In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import ExcelWriter

# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

In [3]:
%matplotlib inline 

In [4]:
init_notebook_mode(connected=True) #this will connect plotly to the java script

In [5]:
cf.go_offline() #this will allow you to use cufflinks offline

# Hourly Data for the HPPs

# The outputs of this script are: 
1. The capacity factor of the river segments in the reference scenario. 
2. The residual capacity of the river segments in the reference scenario. 
3. The weekly distribution of the flow (compare to the respective month)

To be able to convert the montly flow data to weekly flow, we need to know the share of the flow in each week of the month. 

In this part we will use the hourly data for the previous years to generate the weekly shares.

Weekly share = (flow in week_x of month_y)/(total flow of month_y)

How to get the monthly and weekly flow from hourly data?

Monthly flow: groupby('month').mean()

Weekly flow: groupby(['month','week']).mean()


### Reading the file and seting the dataframe

In [18]:
folder_path = os.path.join('Data', 'hydrological_data')
hourly_data = os.path.join(folder_path, 'hourly data 2001-2010 for all river segments 20200930.xlsx')
df = pd.read_excel(hourly_data)

In [19]:
df2 =df.drop('Date', axis=1).copy()

In [20]:
# Replace negetive values with zero then with NaN. 
df2[df2<0]=0
df2= df2.replace({0:np.nan, 0:np.nan})

In [21]:
df2['Date']=df['Date']

In [22]:
df2.rename({'Orhid lake outtflow':'Orhid_CT',
           'Globocica lake side flow':'Globo_CT',
           'Globocica lake outflow':'Globo_HPP',
          'Debar/Shpilje lake side flow':'Shpil_CT',
           'Debar/Shpilje HPP':'Shpil_HPP',
           'Skavica side flow':'Skavi_CT',
           'Skavica outflow':'Skavi_HPP',
           'White Drini outflow':'WDrin_CT',
           'Fierza lake side flow':'Fierz_CT',
           'Fierza lake outflow':'Fjerz_HPP',
           'Koman lake side flow':'Koman_CT',
           'Koman lake outflow':'Koman_HPP',
           'Vau I Dejes lake side flow':'VauDe_CT',
           'Vau I Dejes lake outflow':'VauDe_HPP',
          }, axis=1, inplace=True)
df2.head()

,Orhid_CT,Globo_CT,Globo_HPP,Shpil_CT,Shpil_HPP,Skavi_CT,Skavi_HPP,WDrin_CT,Fierz_CT,Fjerz_HPP,Koman_CT,Koman_HPP,VauDe_CT,VauDe_HPP,Date
0,15.066,12.692,27.758,26.797,54.555,NaN,21.440,17.926,11.024,50.39,16.51,66.90,5.17,72.07,2001-11-01 00:00:00
1,14.975,12.735,27.710,26.704,54.414,NaN,21.412,17.917,11.061,50.39,16.65,67.04,5.00,72.04,2001-11-01 01:00:00
2,14.736,12.878,27.614,26.345,53.959,NaN,21.363,17.902,11.125,50.39,16.61,67.00,4.99,71.99,2001-11-01 02:00:00
3,14.471,13.047,27.518,26.232,53.750,NaN,21.324,17.888,11.178,50.39,16.58,66.97,4.96,71.93,2001-11-01 03:00:00
4,14.311,13.112,27.423,26.120,53.543,NaN,21.282,17.874,11.234,50.39,16.55,66.94,4.94,71.88,2001-11-01 04:00:00


## Sorting the dataframe by years, months and weeks 

In [23]:
dfCT=df2.filter(like='_CT').astype('float')
dfCT['Date']=df2['Date']
dfCT['year'] = dfCT['Date'].dt.year 
dfCT['month'] = dfCT['Date'].dt.month
dfCT['week'] = dfCT['Date'].dt.week
dfCT.head()

,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT,Date,year,month,week
0,15.066,12.692,26.797,NaN,17.926,11.024,16.51,5.17,2001-11-01 00:00:00,2001,11,44
1,14.975,12.735,26.704,NaN,17.917,11.061,16.65,5.00,2001-11-01 01:00:00,2001,11,44
2,14.736,12.878,26.345,NaN,17.902,11.125,16.61,4.99,2001-11-01 02:00:00,2001,11,44
3,14.471,13.047,26.232,NaN,17.888,11.178,16.58,4.96,2001-11-01 03:00:00,2001,11,44
4,14.311,13.112,26.120,NaN,17.874,11.234,16.55,4.94,2001-11-01 04:00:00,2001,11,44


In [24]:
# Since we do not have data for all months in 2001 and 2010, we will delete them to reduce the errors in averaging
# Deleting week 53

dfCT = dfCT[(dfCT.year !=2001) & (dfCT.year != 2010) & (dfCT.week != 53)]


In [89]:
#weeklyflow=dfCT.groupby(['month','week'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
weeklyflow=dfCT.groupby(['week'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
weeklyflow.tail()

<ipython-input-89-52afe050703b>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,week,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT
47,48,24.689499,6.211789,20.644203,54.794622,95.727112,100.325883,152.003678,40.156745
48,49,24.154148,5.663728,19.102630,66.873062,154.678801,104.716652,192.554262,35.190512
49,50,23.482809,5.469980,24.714745,47.108717,130.518612,120.230543,162.282401,43.765810
50,51,26.554226,4.196375,18.982355,39.727583,98.050476,102.989987,110.125885,31.456109
51,52,25.647781,6.085202,24.337142,40.842600,107.916158,92.772008,114.091949,30.934496


## Calculate the capacity factor for all the river segments and save files

In [91]:
# Generating the weekly capacity factor
#CF = weeklyflow/(weeklyflow.max().drop(['month','week']))
CF = weeklyflow/(weeklyflow.max().drop(['week']))
CF['week']=weeklyflow['week']
CF

,Fierz_CT,Globo_CT,Koman_CT,Orhid_CT,Shpil_CT,Skavi_CT,VauDe_CT,WDrin_CT,week
0,0.759800,0.688029,0.419954,0.773768,0.459304,0.439128,0.471493,0.547393,1
1,0.693348,1.000000,0.565452,0.942294,0.643539,0.550697,0.497721,0.672210,2
2,0.848770,0.714529,0.600183,0.879502,0.364144,0.401434,0.639961,0.535525,3
3,0.543659,0.753382,0.627879,0.933417,0.393824,0.525349,0.786844,0.622881,4
4,0.610429,0.944553,0.376030,0.861036,0.430958,0.535114,0.543239,0.571540,5
5,0.560119,0.837276,0.517979,0.779422,0.385228,0.460820,0.495689,0.514268,6
6,0.613334,0.774929,0.572076,0.743286,0.452877,0.443082,0.677106,0.600598,7
7,0.864064,0.636414,0.435704,0.706698,0.523743,0.526712,0.649742,0.716199,8
8,0.872867,0.886506,0.698190,0.790800,0.572027,0.678934,0.643599,0.695654,9
9,0.976929,0.873140,0.629232,0.823332,0.655110,0.858978,0.575469,0.950953,10


In [92]:
CF.columns

Index(['Fierz_CT', 'Globo_CT', 'Koman_CT', 'Orhid_CT', 'Shpil_CT', 'Skavi_CT',
       'VauDe_CT', 'WDrin_CT', 'week'],
      dtype='object')

In [93]:
# Updating the names to match the technologies in the model
CF.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)

In [96]:
# Saving the results for each HPP in a seperate sheet

output_folder = os.path.join('Processed_data', 'Capacity_factors', 'Reference')
os.makedirs(output_folder, exist_ok = True)

names = CF.columns.drop(['week'])

weeklyflow.to_excel(os.path.join(output_folder,'REF_WeeklyFlow.xlsx'),index=False)

for name in names:
    dfname = CF[['week',name]].copy()
    for i in range(20,46):
        dfname['20{}'.format(i)]=CF[name].round(3)
        dfname.drop([name], axis=1).to_excel(os.path.join(output_folder,'REF_CF_{}.xlsx'.format(name)))

# Calculating the residual capacity of the river segment:

In [97]:
#Note that the we are computing the residual capacity based on the average historical flow (groupby...mean()) 
# To get the residual capacity in the wet year you can use (groupby...max()), similliarly (min) can be used to represent the dry year:

df3=dfCT.groupby(['month'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
river_residual_cap = df3.max()*(3600*8760/1000000) #which represnts the max averaege flow
river_residual_cap.round(3)


<ipython-input-97-ab8979b08278>:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



month        378.432
Orhid_CT     958.764
Globo_CT     192.998
Shpil_CT    1179.145
Skavi_CT    1814.542
WDrin_CT    3887.699
Fierz_CT    3278.509
Koman_CT    4525.563
VauDe_CT    1088.983
dtype: float64

In [51]:
river_residual_cap.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)

In [ ]:
# Note that for the river segments the Total Max Capacity and Total Max Capacity should also be set to equal the residual capacity

In [52]:
output_folder = os.path.join('Processed_data', 'Residual_Capacity','Reference')
os.makedirs(output_folder, exist_ok = True)

river_residual_cap.to_excel(os.path.join(output_folder,'REF_Residual_Capacity(MCM)_all_rivers.xlsx')) 


# Weekly flow distribution

In [ ]:
# Reading the file. You can skip this step if the dataframe is open

In [62]:
folder_path = os.path.join('Processed_data', 'Capacity_factors','Reference')
input_data = os.path.join(folder_path, 'REF_WeeklyFlow.xlsx')
weeklyflow = pd.read_excel(input_data)

In [75]:
weeklyflow.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)
weeklyflow.columns

Index(['month', 'week', 'MKCWTLK0BD', 'MKCWTCT1BD', 'MKCWTCT2BD', 'MKCWTCT3DD',
       'XKCWTRS1WD', 'ALCWTCT1DD', 'ALCWTCT2DD', 'ALCWTCT3DD'],
      dtype='object')

In [76]:
weeklyflow_melted = pd.melt(weeklyflow, id_vars=['month','week'],
                     var_name='catchments', value_name='weeklyflow')

In [77]:
weeklyflow_melted.sort_values(['month','week', 'catchments'], inplace=True)

weeklyflow_melted

,month,week,catchments,weeklyflow
360,1,1,ALCWTCT1DD,88.909828
432,1,1,ALCWTCT2DD,84.517336
504,1,1,ALCWTCT3DD,20.205614
72,1,1,MKCWTCT1BD,5.272200
144,1,1,MKCWTCT2BD,21.734347
...,...,...,...,...
143,12,52,MKCWTCT1BD,5.851336
215,12,52,MKCWTCT2BD,23.858567
287,12,52,MKCWTCT3DD,39.432512
71,12,52,MKCWTLK0BD,25.797939


In [78]:
monthlyflow=weeklyflow.groupby(['month'])['MKCWTLK0BD', 'MKCWTCT1BD', 'MKCWTCT2BD', 'MKCWTCT3DD',
       'XKCWTRS1WD', 'ALCWTCT1DD', 'ALCWTCT2DD', 'ALCWTCT3DD'].mean().reset_index()
monthlyflow.head()

<ipython-input-78-d3b108dcbae5>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,month,MKCWTLK0BD,MKCWTCT1BD,MKCWTCT2BD,MKCWTCT3DD,XKCWTRS1WD,ALCWTCT1DD,ALCWTCT2DD,ALCWTCT3DD
0,1,26.637051,8.145247,26.766052,52.036791,130.778819,92.750089,112.659994,40.528937
1,2,25.867279,5.447767,20.708650,34.968789,90.189341,81.071199,98.763550,23.717401
2,3,27.066332,6.109914,33.471026,60.628735,120.123878,112.163835,119.425822,29.456925
3,4,30.408203,4.493874,42.240360,62.483235,75.828858,90.029215,106.938020,23.395539
4,5,29.438032,3.617678,29.099144,34.570765,58.148253,67.324010,93.538894,14.402516


In [79]:
monthlyflow_melted = pd.melt(monthlyflow, id_vars=['month'],
                     var_name='catchments', value_name='monthlyflow')

In [80]:
weeklyflow_melted.sort_values(['month', 'catchments'], inplace=True)
monthlyflow_melted

,month,catchments,monthlyflow
0,1,MKCWTLK0BD,26.637051
1,2,MKCWTLK0BD,25.867279
2,3,MKCWTLK0BD,27.066332
3,4,MKCWTLK0BD,30.408203
4,5,MKCWTLK0BD,29.438032
...,...,...,...
91,8,ALCWTCT3DD,8.156359
92,9,ALCWTCT3DD,15.888488
93,10,ALCWTCT3DD,24.666259
94,11,ALCWTCT3DD,33.629610


In [81]:
dff = pd.merge(weeklyflow_melted, monthlyflow_melted, on=['month','catchments'])

In [82]:
dff.sort_values(['month','week'], inplace=True)

dff['weekpermonth']=dff['weeklyflow']/dff['monthlyflow']
dff.round(3)

,month,week,catchments,weeklyflow,monthlyflow,weekpermonth
0,1,1,ALCWTCT1DD,88.910,92.750,0.959
6,1,1,ALCWTCT2DD,84.517,112.660,0.750
12,1,1,ALCWTCT3DD,20.206,40.529,0.499
18,1,1,MKCWTCT1BD,5.272,8.145,0.647
24,1,1,MKCWTCT2BD,21.734,26.766,0.812
...,...,...,...,...,...,...
551,12,52,MKCWTCT1BD,5.851,6.812,0.859
557,12,52,MKCWTCT2BD,23.859,19.874,1.200
563,12,52,MKCWTCT3DD,39.433,53.163,0.742
569,12,52,MKCWTLK0BD,25.798,24.947,1.034


In [84]:

#dff.loc[(dff['month']==12)&(dff['catchment']=='Fierz_CT')]['weekpermonth'].sum()
# Note that the weekly disctrubution is not supposed to sum up to 1 as we are comparing the weekly average with the monthly average not the monthly sum.

In [85]:
#Writing results
output_folder = os.path.join('Processed_data', 'Weekly_distribution')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'REF_WeeklyDistribution_AllCatchments.xlsx')
writer = pd.ExcelWriter(file_path, engine='xlsxwriter') 
dff.to_excel(writer,index=False)
    
writer.save()

In [137]:
# This is an alternative approach- Do no use it for now

rivers = globo.columns.drop(['Date', 'year', 'month', 'week'])

for river in rivers:
    globo_pvt = pd.pivot_table(globo,
                       values=river,
                       index=['month', 'week'],
                       columns=['year'],
                       aggfunc=np.mean)
    globo_pvt.drop([2001,2010],axis=1, inplace=True)
    #globo_pvt.drop([53], inplace=True)
    globo_pvt.reset_index(inplace=True)
    globo_pvt.fillna(0, inplace=True) #relace all NaN value with zero
    
    globo2= globo_pvt[(globo_pvt!=0).all(axis=1)] #To delete all the rows with zero values
    globo2['weekly_mean'] = globo2.iloc[:,[2,3,4,5,6,7,8,9]].mean(axis=1)
    
    monthly_flow = globo2.groupby(['month']).mean()
    monthly_flow.reset_index(inplace=True)

    monthly_flow['monthly_mean'] = monthly_flow.iloc[:,[2,3,4,5,6,7,8,9]].mean(axis=1)

    df_monthly = monthly_flow[['month','monthly_mean']]
    df_merge = pd.merge(globo2, df_monthly, on='month')
    df_merge['weekly_share'] = df_merge['weekly_mean']/df_merge['monthly_mean']
    dff = df_merge[['month','week','monthly_mean','weekly_mean','weekly_share' ]].copy().round(3)
    #Writing results
    output_folder = os.path.join('Processed_data', 'Weekly_distribution')
    os.makedirs(output_folder, exist_ok = True)

    file_path=os.path.join(output_folder, '{}'.format(river)+'_weekly_share.xlsx')
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter') 
    dff.to_excel(writer)
    
    writer.save()

In [ ]:
#end